<a href="https://colab.research.google.com/github/GenaroHacker/actionable_builder/blob/main/database_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title sql_core
import sqlite3

def create_table( database_name , table_name , table_structure ):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    try:
        my_cursor.execute('''
            CREATE TABLE {} ( {} )
        '''.format(table_name, table_structure))
        my_connection.commit()
        my_connection.close()
        #Table created successfully
    except sqlite3.OperationalError:
        #Table already exists
        pass

def insert_record(database_name, record):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    my_cursor.execute(record)
    my_connection.commit()
    my_connection.close()

def insert_several_records(database_name, multiple_records):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    for i in multiple_records:
        my_cursor.execute(i)
    my_connection.commit()
    my_connection.close()

def read_records(database_name, table_name):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    my_cursor.execute("SELECT * FROM {}".format(table_name))
    records=my_cursor.fetchall()
    my_connection.close()
    return records

def read_last_record(database_name, table_name):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    my_cursor.execute("SELECT * FROM {} ORDER BY ID DESC LIMIT 1".format(table_name))
    records=my_cursor.fetchall()
    my_connection.close()
    return records[0]

def update_record(database_name, record):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    my_cursor.execute(record)
    my_connection.commit()
    my_connection.close()

def remove_record(database_name, record):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    my_cursor.execute(record)
    my_connection.commit()
    my_connection.close()

def run_command(database_name, command):
    my_connection=sqlite3.connect(database_name)
    my_cursor=my_connection.cursor()
    my_cursor.execute(command)
    my_connection.commit()
    my_connection.close()

if __name__ == "__main__":
    #Create one Table
    columns = """
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            ITEM_NAME VARCHAR(50),
            PRICE INTEGER,
            SECTION VARCHAR(20)"""
    create_table("BaseProducts","TableProducts",columns)

    #Insert one record
    insert_record("BaseProducts","INSERT INTO TableProducts VALUES (NULL,'BALL',10,'SPORT')")
    #Record inserted successfully!

    #Insert several records
    insert_several_records("BaseProducts",[
        "INSERT INTO TableProducts VALUES (NULL,'GOLF STICK',25,'SPORT')",
        "INSERT INTO TableProducts VALUES (NULL,'GLASS',20,'CERAMIC')",
        "INSERT INTO TableProducts VALUES (NULL,'T-SHIRT',5,'CLOTHES')"
    ])
    #Records inserted successfully!

    #Read all records
    list_of_tuples = read_records("BaseProducts","TableProducts")

    #Read last record
    last_record = read_last_record("BaseProducts","TableProducts")

    #Update record
    update_record("BaseProducts","UPDATE TableProducts SET ITEM_NAME='BALL NAME UPDATED' WHERE ID=3")
    #The record with ID=3 has been updated successfully!

    #Remove record
    remove_record("BaseProducts","DELETE FROM TableProducts WHERE ID=3")
    #The record with ID=4 has been removed successfully!

In [2]:
# @title Set Up
import sqlite3
from google.colab import files
import os

# Constant for the database name
DATABASE_NAME = 'TaskManagement.db'

def create_table(database_name, table_name, table_structure):
    with sqlite3.connect(database_name) as conn:
        cursor = conn.cursor()
        cursor.execute(f'CREATE TABLE IF NOT EXISTS {table_name} ({table_structure});')
        conn.commit()

def insert_record(database_name, insert_sql, params=()):
    with sqlite3.connect(database_name) as conn:
        cursor = conn.cursor()
        cursor.execute(insert_sql, params)
        conn.commit()
        return cursor.lastrowid

def create_empty_database():
    tables = {
        "PLACES": "ID INTEGER PRIMARY KEY, NAME TEXT UNIQUE",
        "CONTEXTS": "ID INTEGER PRIMARY KEY, PLACE_ID INTEGER, NAME TEXT, RANK INTEGER, "
                    "FOREIGN KEY(PLACE_ID) REFERENCES PLACES(ID), UNIQUE(PLACE_ID, NAME)",
        "TAGS": "ID INTEGER PRIMARY KEY, CONTEXT_ID INTEGER, NAME TEXT, RANK INTEGER, "
                "FOREIGN KEY(CONTEXT_ID) REFERENCES CONTEXTS(ID), UNIQUE(CONTEXT_ID, NAME)",
        "TASKS": "ID INTEGER PRIMARY KEY, TAG_ID INTEGER, NAME TEXT, RANK INTEGER, "
                 "FOREIGN KEY(TAG_ID) REFERENCES TAGS(ID), UNIQUE(TAG_ID, NAME)",
        "STEPS": "ID INTEGER PRIMARY KEY, TASK_ID INTEGER, NAME TEXT, ORDER_SEQUENCE INTEGER, "
                 "MINUTES INTEGER, FOREIGN KEY(TASK_ID) REFERENCES TASKS(ID), UNIQUE(TASK_ID, ORDER_SEQUENCE)",
        "PLACEHOLDERS": "ID INTEGER PRIMARY KEY, TYPE TEXT, VALUE TEXT, UNIQUE(TYPE, VALUE)",
        "HISTORY": "ID INTEGER PRIMARY KEY, STEP_ID INTEGER, ACTION TEXT, TIME TEXT, "
                   "FOREIGN KEY(STEP_ID) REFERENCES STEPS(ID)"
    }
    for table_name, table_structure in tables.items():
        create_table(DATABASE_NAME, table_name, table_structure)

def check_or_insert_with_parent(table, name, parent_id_column, parent_id):
    with sqlite3.connect(DATABASE_NAME) as conn:
        cursor = conn.cursor()

        if parent_id is None:  # Handling for places
            cursor.execute("SELECT ID FROM PLACES WHERE NAME = ?", (name,))
            record = cursor.fetchone()
            if record is None:
                cursor.execute("INSERT INTO PLACES (NAME) VALUES (?)", (name,))
                conn.commit()
                return cursor.lastrowid
            return record[0]
        else:  # Handling for contexts, tags, tasks
            cursor.execute(f"SELECT ID FROM {table} WHERE NAME = ? AND {parent_id_column} = ?", (name, parent_id))
            record = cursor.fetchone()
            if record is None:
                cursor.execute(f"INSERT INTO {table} ({parent_id_column}, NAME, RANK) VALUES (?, ?, NULL)", (parent_id, name))
                conn.commit()
                return cursor.lastrowid
            return record[0]

def check_step_exists(task_id, order_sequence):
    with sqlite3.connect(DATABASE_NAME) as conn:
        cursor = conn.cursor()
        cursor.execute("SELECT COUNT(*) FROM STEPS WHERE TASK_ID = ? AND ORDER_SEQUENCE = ?", (task_id, order_sequence))
        count = cursor.fetchone()[0]
        return count > 0

def add_task(task_input):
    place_id = check_or_insert_with_parent('PLACES', task_input["place"], 'ID', None)
    context_id = check_or_insert_with_parent('CONTEXTS', task_input["context"], 'PLACE_ID', place_id)
    tag_id = check_or_insert_with_parent('TAGS', task_input["tag"], 'CONTEXT_ID', context_id)
    task_id = check_or_insert_with_parent('TASKS', task_input["task"], 'TAG_ID', tag_id)

    for index, step in enumerate(task_input["steps"], start=1):
        if not check_step_exists(task_id, index):
            step_sql = "INSERT INTO STEPS (TASK_ID, NAME, ORDER_SEQUENCE, MINUTES) VALUES (?, ?, ?, NULL)"
            insert_record(DATABASE_NAME, step_sql, (task_id, step, index))

def add_placeholders(placeholders_dict):
    insert_sql = "INSERT INTO PLACEHOLDERS (TYPE, VALUE) VALUES (?, ?)"
    for placeholder_type, values in placeholders_dict.items():
        for value in values:
            try:
                insert_record(DATABASE_NAME, insert_sql, (placeholder_type, value))
            except sqlite3.IntegrityError:
                continue

def initialize_database():
    user_choice = input("Choose an option:\n1. Create empty database\n2. Create database with example data\n3. Upload database\n")
    if user_choice == '1':
        create_empty_database()
    elif user_choice == '2':
        create_example_database()
    elif user_choice == '3':
        uploaded_files = files.upload()
        for filename in uploaded_files.keys():
            os.rename(filename, DATABASE_NAME)
            print(f"Uploaded file '{filename}' has been renamed to '{DATABASE_NAME}'")

def create_example_database():
    create_empty_database()
    # Insert example data into each table

def download_database():
    files.download(DATABASE_NAME)



In [3]:
# @title Main Execution

# Main Execution
if __name__ == "__main__":
    initialize_database()

    # Example task input
    task = {
        "place": "home",
        "context": "create",
        "tag": "guitar",
        "task": "loop %note%",
        "steps": [
            "identify the note in each string",
            "start the metronome at 30",
            "loop over each string 3 times"
        ]
    }

    add_task(task)

    # Example usage of add_placeholders
    placeholders = {
        "note": ["C", "C#", "D"],
        "color": ["red", "blue", "green"]
    }

    add_placeholders(placeholders)

    download_database()

Choose an option:
1. Create empty database
2. Create database with example data
3. Upload database
1


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
# @title Add Example Data
# Task 1
task = {
    "place": "home",
    "context": "exercise",
    "tag": "yoga",
    "task": "complete a %duration%-minute yoga session",
    "steps": [
        "set up a comfortable space with a yoga mat",
        "choose a %duration%-minute yoga routine video",
        "follow the video instructions carefully",
        "focus on breathing and posture",
        "cool down and relax for a few minutes after finishing"
    ]
}
add_task(task)

# Task 2
task = {
    "place": "home",
    "context": "recreate",
    "tag": "painting",
    "task": "paint a landscape using %color% colors",
    "steps": [
        "gather all painting supplies",
        "choose a landscape reference or use imagination",
        "sketch the basic outline on canvas",
        "start painting with %color% colors from the background to foreground",
        "let the painting dry and clean up the workspace"
    ]
}
add_task(task)

# Task 3
task = {
    "place": "home",
    "context": "create",
    "tag": "cooking",
    "task": "bake %flavor% cookies",
    "steps": [
        "preheat the oven and prepare baking sheets",
        "mix dough ingredients and fold in %flavor% chips",
        "scoop dough onto baking sheets",
        "bake until cookies are golden brown",
        "cool cookies on a rack before serving"
    ]
}
add_task(task)

# Task 4
task = {
    "place": "office",
    "context": "work",
    "tag": "report",
    "task": "compile a %period% sales report",
    "steps": [
        "gather all relevant sales data from %period%",
        "analyze trends and key performance indicators",
        "create charts and graphs to represent data",
        "write a summary of findings",
        "format the report and prepare for presentation"
    ]
}
add_task(task)

# Task 5
task = {
    "place": "gym",
    "context": "exercise",
    "tag": "strength training",
    "task": "complete a strength training circuit with %equipment%",
    "steps": [
        "start with a 5-minute warm-up",
        "perform three sets of weight lifting exercises with %equipment%",
        "include exercises targeting major muscle groups",
        "take short breaks between sets",
        "end with a 5-minute cool down and stretching"
    ]
}
add_task(task)

# Task 6
task = {
    "place": "home",
    "context": "exercise",
    "tag": "cardio",
    "task": "perform a %time%-minute high-intensity workout",
    "steps": [
        "start with a quick warm-up",
        "follow a %time%-minute HIIT video",
        "perform each exercise with high energy",
        "ensure breaks are short to maintain intensity",
        "cool down with light stretching"
    ]
}
add_task(task)

# Task 7
task = {
    "place": "home",
    "context": "sleep",
    "tag": "relaxation",
    "task": "prepare for a restful nights sleep",
    "steps": [
        "turn off all electronic devices %time% minutes before bed",
        "prepare a cup of %herb% tea",
        "read a book or practice meditation",
        "ensure the room is dark and at a comfortable temperature",
        "try to go to bed at the same time each night"
    ]
}
add_task(task)

# Task 8
task = {
    "place": "office",
    "context": "work",
    "tag": "meeting",
    "task": "organize a %duration%-minute team meeting",
    "steps": [
        "prepare the meeting agenda in advance",
        "invite all relevant team members",
        "start the meeting on time and follow the agenda",
        "encourage team interaction and discussion",
        "summarize key points and conclude on time"
    ]
}
add_task(task)

# Task 9
task = {
    "place": "gym",
    "context": "exercise",
    "tag": "cardio",
    "task": "run on the treadmill for %time% minutes",
    "steps": [
        "start with a 5-minute warm-up at a slow pace",
        "increase speed to a comfortable run",
        "maintain steady pace for %time% minutes",
        "cool down with a 5-minute slow walk",
        "stretch after finishing the run"
    ]
}
add_task(task)

# Task 10
task = {
    "place": "home",
    "context": "create",
    "tag": "crafting",
    "task": "create a %item% using %material%",
    "steps": [
        "gather all necessary tools and materials",
        "follow a guide or tutorial for making %item%",
        "carefully measure and cut %material%",
        "assemble and secure all parts of %item%",
        "finalize with painting or decoration as needed"
    ]
}
add_task(task)

# Task 11
task = {
    "place": "home",
    "context": "recreate",
    "tag": "music",
    "task": "practice %instrument% for %time% minutes",
    "steps": [
        "set up your %instrument% in a quiet space",
        "tune the %instrument% before starting",
        "practice scales or specific pieces for %time% minutes",
        "focus on technique and rhythm",
        "store the %instrument% safely after practice"
    ]
}
add_task(task)

# Task 12
task = {
    "place": "office",
    "context": "work",
    "tag": "organization",
    "task": "organize and label %item% files",
    "steps": [
        "gather all %item% files",
        "sort the files by date or relevance",
        "label each file clearly",
        "store the files in designated cabinets or folders",
        "update the digital tracking system if available"
    ]
}
add_task(task)

# Task 13
task = {
    "place": "home",
    "context": "exercise",
    "tag": "fitness",
    "task": "do a %time%-minute full-body workout",
    "steps": [
        "choose a %time%-minute workout routine",
        "set up your workout space with necessary equipment",
        "follow the routine, focusing on form",
        "take brief rests between exercises if needed",
        "stretch and cool down after the workout"
    ]
}
add_task(task)

# Task 14
task = {
    "place": "gym",
    "context": "exercise",
    "tag": "flexibility",
    "task": "attend a %duration%-minute stretching class",
    "steps": [
        "arrive at the gym early to secure a spot",
        "bring a yoga mat and water bottle",
        "participate actively in the class",
        "focus on breathing and stretching each muscle group",
        "hydrate and rest after the class"
    ]
}
add_task(task)

# Task 15
task = {
    "place": "office",
    "context": "work",
    "tag": "research",
    "task": "conduct research on %topic%",
    "steps": [
        "define the scope of research on %topic%",
        "gather resources and materials",
        "take detailed notes and organize information",
        "analyze findings and draw conclusions",
        "compile a report or presentation on the research"
    ]
}
add_task(task)

# Task 16
task = {
    "place": "home",
    "context": "create",
    "tag": "gardening",
    "task": "plant %plant% in the garden",
    "steps": [
        "choose a suitable location for %plant%",
        "prepare the soil and dig holes",
        "plant %plant% and water thoroughly",
        "add mulch around the plant for moisture retention",
        "regularly check and care for %plant%"
    ]
}
add_task(task)

# Task 17
task = {
    "place": "gym",
    "context": "exercise",
    "tag": "cardio",
    "task": "complete a %time%-minute cycling class",
    "steps": [
        "set up the bike to fit your height",
        "start with a light warm-up",
        "follow the instructors routine",
        "maintain a steady pace and intensity",
        "cool down and stretch after the class"
    ]
}
add_task(task)

# Task 18
task = {
    "place": "home",
    "context": "sleep",
    "tag": "relaxation",
    "task": "perform %time% minutes of bedtime yoga",
    "steps": [
        "create a calm atmosphere in your bedroom",
        "choose a gentle %time%-minute yoga routine",
        "focus on relaxing poses and deep breathing",
        "end with a meditation or mindfulness exercise",
        "prepare for a peaceful nights sleep"
    ]
}
add_task(task)

# Task 19
task = {
    "place": "office",
    "context": "work",
    "tag": "planning",
    "task": "plan the agenda for %event%",
    "steps": [
        "define the objectives of %event%",
        "list key topics and speakers",
        "allocate time slots for each session",
        "incorporate breaks and networking opportunities",
        "circulate the agenda to participants in advance"
    ]
}
add_task(task)

# Task 20
task = {
    "place": "home",
    "context": "recreate",
    "tag": "reading",
    "task": "read %book% for %time% minutes",
    "steps": [
        "find a quiet and comfortable reading spot",
        "set a timer for %time% minutes",
        "focus on reading and understanding %book%",
        "take brief notes if necessary",
        "store the book safely after reading"
    ]
}
add_task(task)

placeholders = {
    "duration": ["20", "30", "45"],
    "color": ["red", "blue", "green"],
    "flavor": ["chocolate chip", "peanut butter", "oatmeal raisin"],
    "period": ["monthly", "quarterly", "yearly"],
    "equipment": ["dumbbells", "kettlebells", "resistance bands"],
    "time": ["15", "30", "45"],
    "herb": ["chamomile", "lavender", "mint"],
    "item": ["photo frame", "jewelry box", "birdhouse"],
    "material": ["wood", "cardboard", "metal"],
    "instrument": ["guitar", "piano", "violin"],
    "topic": ["market trends", "new technologies", "consumer behavior"],
    "plant": ["tomatoes", "roses", "lavender"],
    "event": ["team meeting", "product launch", "workshop"],
    "book": ["a novel", "a biography", "a self-help guide"]
}

add_placeholders(placeholders)
